In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [3]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()

    # Initialise the info dict
    info_dict = {}
    info_dict['bboxes'] = []

    # Parse the XML Tree
    for elem in root:
        # Get the file name
        if elem.tag == "filename":
            info_dict['filename'] = elem.text

        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                if subelem.text is not None: # .tag != "depth": 
                    image_size.append(int(subelem.text))

            info_dict['image_size'] = tuple(image_size)

        # Get details of the bounding box
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text

                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(float(subsubelem.text))
            info_dict['bboxes'].append(bbox)

    return info_dict

In [4]:
print(extract_info_from_xml('Norway/train/annotations/xmls/Norway_000000.xml'))

{'bboxes': [{'class': 'D00', 'xmin': 1138, 'ymin': 1281, 'xmax': 1169, 'ymax': 1336}, {'class': 'D20', 'xmin': 1537, 'ymin': 1131, 'xmax': 1629, 'ymax': 1247}, {'class': 'D00', 'xmin': 1773, 'ymin': 1825, 'xmax': 1862, 'ymax': 2038}, {'class': 'D00', 'xmin': 1589, 'ymin': 1296, 'xmax': 1624, 'ymax': 1343}, {'class': 'D00', 'xmin': 1507, 'ymin': 1216, 'xmax': 1527, 'ymax': 1254}], 'filename': 'Norway_000000.jpg', 'image_size': (3650, 2044)}


In [5]:
# Dictionary that maps class names to IDs
class_name_to_id_mapping = {"D00": 1,
                           "D10": 2,
                           "D20": 3,
                           "D40": 4}

# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(info_dict):
    print_buffer = []

    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = (b["xmin"] + b["xmax"]) / 2
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])

        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width    /= image_w
        b_height   /= image_h

        #Write the bbox details to the file
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))

    # Name of the file which we have to save
    save_file_name = os.path.join("Norway/train/annotations/xmls", info_dict["filename"].replace("jpg", "txt"))

    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# Get the annotations
annotations = [os.path.join('Norway/train/annotations/xmls', x) for x in os.listdir('Norway/train/annotations/xmls') if x[-3:] == "xml"]
annotations.sort()

# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov5(info_dict)
annotations = [os.path.join('Norway/train/annotations/xmls', x) for x in os.listdir('Norway/train/annotations/xmls') if x[-3:] == "txt"]

  1%|          | 47/8161 [01:30<3:15:45,  1.45s/it] 

In [ ]:
annotations

In [ ]:
#annotations = [os.path.join('labels/train', x) for x in os.listdir('labels/train') if x[-3:] == "txt"]

In [ ]:
#random.seed(20)

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    #print(len(annotation_list))
    if len(annotation_list[0]) < 4:
        plotted_image = ImageDraw.Draw(image)
    else:
        plotted_image = ImageDraw.Draw(image)
        transformed_annotations = np.copy(annotations)
        #print(transformed_annotations)
        transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
        transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h
        #print(transformed_annotations)

        transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
        transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
        transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
        transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]
        #print(transformed_annotations)

        for ann in transformed_annotations:
            obj_cls, x0, y0, x1, y1 = ann
            plotted_image.rectangle(((x0,y0), (x1,y1)), width = 10, outline ="red")

            plotted_image.text((x0, y0 - 10), class_id_to_name_mapping[(int(obj_cls))])

    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file
annotation_file = random.choice(annotations)
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) if y != "" else 0 for y in x ] for x in annotation_list]
    
#Get the corresponding image file
image_file = annotation_file.replace("labels", "images").replace("txt", "jpg")
print(image_file)
print(annotation_file)
assert os.path.exists(image_file)

#Load the image
image = Image.open(image_file)

#Plot the Bounding Box
#print(annotation_list)
plot_bounding_box(image, annotation_list)


In [ ]:
# Read images and annotations
images = [os.path.join('Norway/train/images/train', x) for x in os.listdir('Norway/train/images/train')]
annotations = [os.path.join('Norway/train/annotations/xmls', x) for x in os.listdir('Norway/train/annotations/xmls') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)


In [ ]:
train_images

In [ ]:
!mkdir images labels

In [ ]:
!mkdir images/train images/val images/test labels/train labels/val labels/test

In [ ]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')
#!mv annotations labels
#%cd ../